# Temporal Clustering

This has been inspired by a variety of bits. Most notably [1](https://arxiv.org/pdf/1606.04695.pdf) which used attention mechanisms and "replanning points" to learn sequences of actions. Most key to the inspiration this gave is that temporal convolution could be used. Also the increment operator and attention mechanisms could be useful.


I do need to learn about transformer networks. Its possible there is a relationship there.

There are a couple key points to this (elaborate)

- if the window is used as a convolution over time this is essentially a interesting ngram finder    
- the key component of EM algorithm is that those modes that are activated become updated.    
- we can make this deep with feedback to correct a variety of errors    
- attention mechanism of which pieces of the mode should be activated next could be used with some fuzziness. lets say the mean was "test" if we have seen "te" then the attention mechanism would weight the importance of the "s" higher. if a "t" was found next it might still improve activation but less so then if the "s" was found but more then if the "a" was found.    


***
1) [Vezhnevets, A., Mnih, V., Osindero, S., Graves, A., Vinyals, O., & Agapiou, J. (2016). Strategic attentive writer for learning macro-actions. In Advances in neural information processing systems (pp. 3486-3494).](https://arxiv.org/pdf/1606.04695.pdf)    
2) [Gutenburg Project](http://www.gutenberg.org)

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.distributions import Normal, MultivariateNormal, Categorical

import string
from random import *
import re
import math
import time

import subprocess, os, glob
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

device = 'cuda:5'
print (device)

print (os.getcwd())

print(torch.__version__)

# from multimodal import MultiModalClassification
# from GMM import *
from dennytools.silence import *


cuda:5
/home/tbjackso/NeoNeo
1.2.0


### Dataset

I want to work with some true sequence data so I'm going to work with text. In particular stories from the Project Gutenburg [2](http://www.gutenberg.org) in this case Alice in wonderland. I actually want to experiment with a few traditional RNNs too so lets get to work. the first goal is that we have to create a dataset out of the text which i have saved as Alice.txt. We must strip the header and footer. Everything else can stay i think. But then we have to convert it into onehot encoded by letters (i'm not doing word to vec here) and then the dataset must return snippets of text from various locations of a specified length. How am I going to do that? I don't really know.... The easiest would be to load it as a 1D vector of numbers I think and then one hot encode and finally trim reshape and/or use a convolution to create window snippets. Note we should probably pad with empty space

In [67]:
text = ''
header = True
headerstr = '^\*\*\* START OF THIS PROJECT GUTENBERG'
footerstr = '^\*\*\* END OF THIS PROJECT GUTENBERG'

def replace(string, substitutions):

    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile('|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

preprocess = {chr(8216):"'", chr(8217):"'", chr(8220):"'", chr(8221):"'"}
print (preprocess)

linecount = 0
for line in open('Alice.txt'):
#     print ("-----------")
#     print (line)
#     print (line.encode().decode('ascii', 'replace'))
    
    if (header):
        # check if end of header is found and switch header off
        if (re.search(headerstr, line) is not None):
            header = False
    else:
        # check if start of footer is found and stop if so otherwise add line to text
        if (re.search(footerstr, line) is not None):
            break
        else:
            linecount += 1
            
            text += replace(line, preprocess)#.decode('ascii')#.replace('\n', ' ')
print (len(text))
print (linecount)
# print (text)

# Now lets convert into something useful
full = [chr(i) for i in range(128)]
print(''.join(full))
unique = sorted(list(set(text)))
print (len(unique))
print (unique)

print ("++++++++++++++++++++++++")
print ([u for u in unique if u not in string.printable])
print ([ord(u) for u in unique if u not in full])
print ("------------------------")

import string
print (string.printable)
print (len(string.printable))
alpha = string.printable
l2n = {a:i for (i,a) in enumerate(alpha)}
n2l = alpha



tststr = "hello World! []()"
tstnum = list(map(lambda a:l2n[a], tststr))
print (tststr)
print (tstnum)
bckstr = ''.join(list(map(lambda n: alpha[n], tstnum)))
print (bckstr)


{'‘': "'", '’': "'", '“': "'", '”': "'"}
144530
3356
 	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~
70
['\n', ' ', '!', "'", '(', ')', '*', ',', '-', '.', '0', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
++++++++++++++++++++++++
[]
[]
------------------------
0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

100
hello World! []()
[17, 14, 21, 21, 24, 94, 58, 24, 27, 21, 13, 62, 94, 84, 86, 69, 70]
hello World! []()


In [57]:
class ConvLN():
    def __init__(self, alpha=string.printable):
        self.alpha = alpha
        self.dict = {a:i for (i,a) in enumerate(alpha)}
        self.size = len(alpha) + 1
    
    def l2n(self, a):
        try:
            n = self.dict[a]
        except KeyError:
            print ("unknown letter: {}".format(a))
            print (ord(a))
            n = len(self.alpha)
        return n
    
    def s2n(self, s):
        return [self.l2n(a) for a in s]
    
    def s2t(self, s, l):
        ls = len(s)
        r = (l-(ls%l))
        d = ls + r
        
        nums = torch.tensor(self.s2n(s), dtype=torch.long).unsqueeze(1)
        ten = torch.zeros((ls, self.size))
        out = torch.zeros((d, self.size))
        ten = ten.scatter_(1, nums, 1)
        out[:ls] = ten
        out[ls:] = ten[:r]
        out = out.view(-1, l, self.size)
        return out
            
conv = ConvLN('abc')
print (conv.alpha)
print (conv.dict)
print (conv.size)
print (conv.l2n('a'))
print (conv.l2n('d'))
print (conv.s2n('abcdefg'))

print (conv.s2t('abcdefg', 10).shape)
print (conv.s2t('abcdefg', 10))

abc
{'a': 0, 'b': 1, 'c': 2}
4
0
unknown letter: d
100
3
unknown letter: d
100
unknown letter: e
101
unknown letter: f
102
unknown letter: g
103
[0, 1, 2, 3, 3, 3, 3]
unknown letter: d
100
unknown letter: e
101
unknown letter: f
102
unknown letter: g
103
torch.Size([1, 10, 4])
unknown letter: d
100
unknown letter: e
101
unknown letter: f
102
unknown letter: g
103
tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]]])


In [68]:
conv = ConvLN()

data = conv.s2n(text)
print (data[:100])

data = conv.s2t(text, len(text)).long()
print (data[0,10:15])
print (conv.size)

[96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 36, 47, 44, 38, 40, 68, 54, 94, 36, 39, 57, 40, 49, 55, 56, 53, 40, 54, 94, 44, 49, 94, 58, 50, 49, 39, 40, 53, 47, 36, 49, 39, 96, 96, 47, 14, 32, 18, 28, 94, 38, 10, 27, 27, 24, 21, 21, 96, 96, 55, 43, 40, 94, 48, 44, 47, 47, 40, 49, 49, 44, 56, 48, 94, 41, 56, 47, 38, 53, 56, 48, 94, 40, 39, 44, 55, 44, 50, 49, 94, 3, 75, 0, 96, 96, 96, 96, 96, 38, 43]
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         

# Now onto some more interesting things

To learn sequence data we can learn a probability model p(x1, x2,..., xn) and then make it generative by conditioning, i.e p(x1,...,xn)/p(x1,...,xn-1). But this conditioning is potentially impractical.  